In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np
from scipy.sparse import csr_matrix
import jieba
import jieba.analyse

In [2]:
data_in = "./data/merge_base.data"
output_user_fea = "./data/user_feature_demo.data"
output_item_fea = './data/item_feature_demo.data'
output_itemid_to_name_file = './data/name_id_demo.dict'

# 用户id 0，物品id 1，观看时长 2，？3，性别4，年龄段5，工资6，地址7，物品标题 8，总时长9，物品所在10
# 初始化数据，区分数据
user_fea_list = []
item_fea_list = []
label_list = []
item_name2id = {}
item_id2name = {}

with open(data_in, mode='r', encoding='UTF-8') as fd:
    for line in fd:
        data = line.strip().split("\001")
        if (len(data) != 13):
            continue
        user_id = data[0].strip()
        gender = data[4].strip()
        age = data[5].strip()
        item_id = data[1].strip()
        item_title = data[8].strip()
        watching_time = float(data[2])
        total_time = float(data[10])

        item_name2id[item_title] = item_id
        item_id2name[item_id] = item_title

        user_fea_list.append([user_id, gender, age])
        item_fea_list.append([item_id, item_title])

        score = watching_time / total_time
        label_list.append(score)

In [4]:
# 数据转为计算机能够理解的数据
# 男、女转为0、1
# 年龄段转为 0-18=2 19-25=3 26-35=4 36-45=5 46-100=6
b = 2
for user in user_fea_list:
    # gender
    if user[1] == '女':
        user[1] = 1
    else:
        user[1] = 0

    # age
    if user[2] == '0-18':
        user[2] = 0 + b
    elif user[2] == '19-25':
        user[2] = 1 + b
    elif user[2] == '26-34':
        user[2] = 2 + b
    elif user[2] == '36-45':
        user[2] = 3 + b
    else:
        user[2] = 4 + b


In [39]:
output_user_fea = "./data/user_feature.data"
user_fea_list
with open (output_user_fea, 'w', encoding='UTF-8') as fd:
    weight = '1'
    for line in user_fea_list:
        user_id = line[0]
        gender = str(line[1]) + ":" + weight
        age = str(line[2]) + ":" + weight
        fd.write("\t".join([user_id, gender + " " + age]))
        fd.write("\n")

In [5]:
# 输出用户特征
with open(output_user_fea, 'w', encoding='UTF-8') as fd:
    weight = '1'
    for line in user_fea_list:
        user_id = line[0]
        gender = str(line[1]) + ":" + weight
        age = str(line[2]) + ":" + weight
        fd.write("\t".join([user_id, gender + " " + age]))
        fd.write("\n")

In [6]:
# 输出item id 转 item
with open(output_itemid_to_name_file, mode='w', encoding='UTF-8') as fd:
    for k, v in item_id2name.items():
        fd.write("\t".join([k, v]))
        fd.write("\n")

In [ ]:
# 把词都转为计算机编码 
# 方法一：用set集合存储所有词，然后循环1000开始每次+1作为词的编码
# 方法二：用dict，key为词，如果key不存在则key的value为1000，每次+1.

In [ ]:
# 输出item特征
token_set = set()
word_id_list = []

for item in item_fea_list:
    item_id = item[0]
    item_name = item[1]

    # todo 要不要直接优化成item[1]???
    # todo 把word_id_list改为 dict key为词，value为id，通过词分配id
    word_tfidf = jieba.analyse.extract_tags(item_name, withWeight=True)
    for word, tfidf in word_tfidf:
        token_set.add(word)

    #把词语转为数字：
    for i in enumerate(list(token_set), start=0):
        word_id_list.append(i)
    


# 1.不重复的词语集合
# 2.遍历集合，从1000开始每个词语+1，作为词的id，保存到本地文件。
# 3.把原来的中文词语替换成id

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/1z/sh527w893x91w_6bdtydgxp80000gn/T/jieba.cache
Loading model cost 1.038 seconds.
Prefix dict has been built succesfully.


In [ ]:
for word, tfidf in word_tfidf:
    word = word_id_dict[word]

item[1] = word_tfidf